In [1]:
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options 
import time 

import os
import csv
import requests
import lxml
import time
# import cloudscraper
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from transliterate import slugify
from tqdm import tqdm

In [38]:
with open('download.xlsx', "rb") as f:
    df = pd.read_excel(f, dtype=str)
print(df.shape)
df.head()

(476, 4)


,_MODEL_,_SKU_,Name,Url
0,00000000065,00065,Ведро 9л оцин.,https://www.vseinstrumenti.ru/sadovaya-tehnika...
1,00000000066,00066,Ведро 12л оцин.,https://www.vseinstrumenti.ru/sadovaya-tehnika...
2,00000009200,009200,Ведро 12л стр-ое пластик.,https://www.vseinstrumenti.ru/sadovaya-tehnika...
3,00000002366,002366,Ведро 15л оцин.,https://www.vseinstrumenti.ru/sadovaya-tehnika...
4,00000000068,00068,Ведро 16л стр-ое пластик.,https://www.vseinstrumenti.ru/sadovaya-tehnika...


In [39]:
df1 = pd.DataFrame(columns=['_MODEL_', '_SKU_', '_NAME_', '_CATEGORY_', '_MAIN_CATEGORY_', '_MANUFACTURER_', '_DESCRIPTION_', '_ATTRIBUTES_', '_PRODUCT_IMAGES_', '_STATUS_', '_SEO_KEYWORD_'])
df1.to_excel("opencart.xlsx", index = False)

df1 = pd.DataFrame(columns=['_MODEL_', '_SKU_', 'Name', 'Url', 'Where', 'status_code', 'reason'])
df1.to_excel("error.xlsx", index = False)

In [40]:
for index in tqdm(df.index):
    options = Options()
    options.add_argument('--headless')
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36")
    driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))
    driver.get(df['Url'][index])
    
    if(driver.title.find('Ошибка 404')==-1):
        soup = BeautifulSoup(driver.page_source, "lxml")

        _NAME_ = soup.find('h1').text.strip()
        _SEO_KEYWORD_ = slugify(_NAME_)

        breadcrumbs = soup.find('nav', class_="breadcrumbs").text.split('/')
        breadcrumbs = [s.strip() for s in breadcrumbs]
        _CATEGORY_ = '|'.join(map(str, breadcrumbs[1:]))
        _MAIN_CATEGORY_ = _CATEGORY_
        try:
            _MANUFACTURER_ = soup.find('img', class_="brand-logo")['alt']
        except:
            _MANUFACTURER_ = ""
        try:
            _DESCRIPTION_ = soup.find(itemprop="description").text.strip()
        except:
            _DESCRIPTION_ = ""

        # Find attributes
        tag_names = soup.find(class_='specification-list').findAll(itemprop='name')
        tag_values = soup.find(class_='specification-list').findAll(itemprop='value')
        _ATTRIBUTES_ = ''
        for i in range(len(tag_names)):
            _ATTRIBUTES_ = _ATTRIBUTES_ + 'Характеристики' + '|' + tag_names[i].text.strip() + '|' + tag_values[i].text.strip() + '\n'
        
        # Find images link
        images = soup.find(class_='carousel -vertical').find_all('a')
        big_img = []
        for img in images:
            big_img.append(img['href'])
        
        # Download images
        local_path = []
        for url_img in big_img:
            resp = requests.get(url_img)
            path = 'catalog/' + '/'.join(map(str, url_img.split('/')[4:]))
            local_path.append(path)
            # Save the image
            if (type(resp)!=int and resp.status_code == 200):
                os.makedirs(os.path.dirname(path), exist_ok=True)
                with open(path, "wb") as f:
                    f.write(resp.content)
            else:
                df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Images', resp.status_code, resp.reason]])
                with pd.ExcelWriter('error.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
                    df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)
            
            _PRODUCT_IMAGES_ = ','.join(map(str, local_path))
        
        _STATUS_ = '1'
        
        df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], _NAME_, _CATEGORY_, _MAIN_CATEGORY_, _MANUFACTURER_, _DESCRIPTION_, _ATTRIBUTES_, _PRODUCT_IMAGES_, _STATUS_, _SEO_KEYWORD_]])
        with pd.ExcelWriter('opencart.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
            df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)
    else:
        df1 = pd.DataFrame([[df['_MODEL_'][index], df['_SKU_'][index], df['Name'][index],  df['Url'][index], 'Url', 'response.status_code', 'response.reason']])
        with pd.ExcelWriter('error.xlsx', mode='a', if_sheet_exists="overlay") as writer:  
            df1.to_excel(writer, startrow =writer.sheets['Sheet1'].max_row, index = False, header = False)

100%|██████████| 476/476 [1:09:06<00:00,  8.71s/it]


In [ ]:
driver.quit()